In [2]:
import abc
import warnings
import numpy as np
import torch
import os
import typing
import numpy as np
import torch
import torch.optim
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, average_precision_score
from torch import nn
from torch.nn import functional as F
from tqdm import trange
from torch.distributions import Normal
from torch import distributions as dis

class ParameterDistribution(torch.nn.Module, metaclass=abc.ABCMeta):
    """
    Abstract class that models a distribution over model parameters,
    usable for Bayes by backprop.
    You can implement this class using any distribution you want
    and try out different priors and variational posteriors.
    All torch.nn.Parameter that you add in the __init__ method of this class
    will automatically be registered and know to PyTorch.
    """

    def __init__(self):
        super().__init__()

    @abc.abstractmethod
    def log_likelihood(self, values: torch.Tensor) -> torch.Tensor:
        """
        Calculate the log-likelihood of the given values
        :param values: Values to calculate the log-likelihood on
        :return: Log-likelihood
        """
        pass

    @abc.abstractmethod
    def sample(self) -> torch.Tensor:
        """
        Sample from this distribution.
        Note that you only need to implement this method for variational posteriors, not priors.

        :return: Sample from this distribution. The sample shape depends on your semantics.
        """
        pass

    def forward(self, values: torch.Tensor) -> torch.Tensor:
        # DO NOT USE THIS METHOD
        # We only implement it since torch.nn.Module requires a forward method
        warnings.warn('ParameterDistribution should not be called! Use its explicit methods!')
        return self.log_likelihood(values)

In [15]:
class UniveriateGaussianPrior(ParameterDistribution):
    """
    Univeriate Guassian distribution"
    """
    def __init__(self, mu, sigma):
        super(UniveriateGaussianPrior, self).__init__()
        self.mu = mu
        self.sigma = sigma # sigma is the standard deviation here, not the variance!
    
    def log_likelihood(self, values: torch.Tensor) -> torch.Tensor:
        dist = Normal(loc=self.mu, scale=self.sigma)
        log_likelihood = dist.log_prob(values).sum()
        return log_likelihood

    def sample(self):
       return Normal(loc=self.mu, scale=self.sigma).sample()

In [ ]:
class MultivariateDiagonalGaussian(ParameterDistribution):
    """
    Multivariate diagonal Gaussian distribution,
    i.e., assumes all elements to be independent Gaussians
    but with different means and standard deviations.
    This parameterizes the standard deviation via a parameter rho as
    sigma = softplus(rho).
    """

    def __init__(self, mu: torch.Tensor, rho: torch.Tensor):
        super(MultivariateDiagonalGaussian, self).__init__()  # always make sure to include the super-class init call!
        assert mu.size() == rho.size()
        self.mu = mu
        self.rho = rho
        self.sig = (F.softplus(rho)*0.05 + 1e-5).detach()

    def log_likelihood(self, values: torch.Tensor) -> torch.Tensor:

        log_likelihood = Normal(self.mu, self.sig).log_prob(values).sum()
        return log_likelihood

    def sample(self) -> torch.Tensor:
        epsilon = torch.distributions.Normal(0,1).sample(self.rho.size())
        return self.mu + self.sig*epsilon


In [ ]:
class GaussianMixturePrior(ParameterDistribution):
    """
    Mixture of two Gaussian distributions as described in Bludell et al., 2015.
    """
    def __init__(self, mu_0: torch.Tensor, sigma_0: torch.Tensor, mu_1: torch.Tensor, sigma_1: torch.Tensor, pi: torch.Tensor):
        super(GaussianMixturePrior, self).__init__()  # always make sure to include the super-class init call!
        self.mu_0 = mu_0 # mean of distribution 0
        self.sigma_0 = sigma_0 # std of distrinution 0
        self.mu_1 = mu_1 # mean of distribution 1
        self.sigma_1 = sigma_1 # std of distribution 1
        self.pi = pi # Probabilistic weight

    def log_likelihood(self, values: torch.Tensor) -> torch.Tensor:
        dist_0 = Normal(loc=self.mu_0, scale=self.sigma_0)
        dist_1 = Normal(loc=self.mu_1, scale=self.sigma_1)
        ll_0 = dist_0.log_prob(values)
        ll_1 = dist_1.log_prob(values)
        return torch.log(self.pi * torch.exp(ll_0) + (1 - self.pi) * torch.exp(ll_1)).sum()

    def sample(self) -> torch.Tensor:
        if np.random.rand() < self.pi:
            return Normal(loc=self.mu_0, scale=self.sigma_0).sample()
        else:
            return Normal(loc=self.mu_1, scale=self.sigma_1).sample()

In [4]:
class BayesianLayer(nn.Module):
    """
    Module implementing a single Bayesian feedforward layer.
    It maintains a prior and variational posterior for the weights (and biases)
    and uses sampling to approximate the gradients via Bayes by backprop.
    """
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        """
        Create a BayesianLayer.

        :param in_features: Number of input features
        :param out_features: Number of output features
        :param bias: If true, use a bias term (i.e., affine instead of linear transformation)
        """
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.use_bias = bias

        # Set hyper priors
        mu_0_prior =  torch.tensor(0.0)
        sigma_0_prior = torch.tensor(0.368)
        mu_1_prior = torch.tensor(0.0)
        sigma_1_prior = torch.tensor(0.00091)
        pi_prior = torch.tensor(0.5)

        # Define prior distribution
        self.prior = GaussianMixturePrior(mu_0 = mu_0_prior,
                                          sigma_0 = sigma_0_prior,
                                          mu_1 = mu_1_prior,
                                          sigma_1 = sigma_1_prior,
                                          pi = pi_prior
        )
        assert isinstance(self.prior, ParameterDistribution)
        assert not any(True for _ in self.prior.parameters()), 'Prior SHOULD NOT have parameters'

        # Set intitial hyper poteriors
        std_mu_init = torch.tensor(0.1) # Mean
        std_rho_init = torch.tensor(1.) # Parameterisation of Std
        
        # Initialize weights by sampling from normal distributions
        w_mu_init = Normal(torch.tensor(0.), std_mu_init).sample((out_features, in_features))
        w_rho_init = Normal(torch.tensor(0.), std_rho_init).sample((out_features, in_features))

        # Create parameter distributions
        self.weights_var_posterior = MultivariateDiagonalGaussian(
            mu = torch.nn.Parameter(w_mu_init),
            rho = torch.nn.Parameter(w_rho_init)
        )

        # Error check
        assert isinstance(self.weights_var_posterior, ParameterDistribution)
        assert any(True for _ in self.weights_var_posterior.parameters()), 'Weight posterior must have parameters'

        if self.use_bias:
            # TODO: As for the weights, create the bias variational posterior instance here.
            #  Make sure to follow the same rules as for the weight variational posterior.

            b_mu_init = Normal(torch.tensor(0.), std_mu_init).sample((out_features,))
            b_rho_init = Normal(torch.tensor(0.), std_rho_init).sample((out_features,))

            self.bias_var_posterior = MultivariateDiagonalGaussian(
            mu = torch.nn.Parameter(b_mu_init),
            rho = torch.nn.Parameter(b_rho_init)
        )
            assert isinstance(self.bias_var_posterior, ParameterDistribution)
            assert any(True for _ in self.bias_var_posterior.parameters()), 'Bias posterior must have parameters'
        else:
            self.bias_var_posterior = None

    def forward(self, inputs: torch.Tensor):
        """
        Perform one forward pass through this layer.
        """
        # Sample the weights
        weights = self.weights_var_posterior.sample()

        # Generate the log-liklihood of the prior and log-posterior
        log_prior = self.prior.log_likelihood(weights)
        log_variational_posterior = self.weights_var_posterior.log_likelihood(weights)

        if self.use_bias:
            # Sample the bias posterios and get prior log-likelihood
            bias = self.bias_var_posterior.sample()
            log_prior += self.prior.log_likelihood(bias)
            # Add on the terms to the variatinoal posterior
            log_variational_posterior += self.bias_var_posterior.log_likelihood(bias)
        else:
            bias = None
        # Call low level linear alg operation
        return F.linear(inputs, weights, bias), log_prior, log_variational_posterior

In [ ]:
class BayesNet(nn.Module):
    """
    Module implementing a Bayesian feedforward neural network using BayesianLayer objects.
    """

    def __init__(self, in_features: int, hidden_features: typing.Tuple[int, ...], out_features: int):
        """
        Create a BNN.

        :param in_features: Number of input features
        :param hidden_features: Tuple where each entry corresponds to a (Bayesian) hidden layer with
            the corresponding number of features.
        :param out_features: Number of output features
        """

        super().__init__()

        feature_sizes = (in_features,) + hidden_features + (out_features,)
        num_affine_maps = len(feature_sizes) - 1
        self.layers = nn.ModuleList([
            BayesianLayer(feature_sizes[idx], feature_sizes[idx + 1], bias=True)
            for idx in range(num_affine_maps)
        ])
        self.activation = nn.ReLU()

    def forward(self, x: torch.Tensor) -> typing.Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Perform one forward pass through the BNN using a single set of weights
        sampled from the variational posterior.

        :param x: Input features, float tensor of shape (batch_size, in_features)
        :return: 3-tuple containing
            i) output features using stochastic weights from the variational posterior,
            ii) sample of the log-prior probability, and
            iii) sample of the log-variational-posterior probability
        """

        log_prior = torch.tensor(0.0)
        log_variational_posterior = torch.tensor(0.0)

        for idx, current_layer in enumerate(self.layers):
            x, log_prior_layer, log_variational_posterior_layer = current_layer(x)
            if idx < len(self.layers) - 1:
                x = self.activation(x)

            log_prior += log_prior_layer
            log_variational_posterior += log_variational_posterior_layer

        """
        Summing up the log_variational_posterior across all layers is necessary because the 
        variational inference process aims to minimize the Kullback-Leibler (KL) divergence between the variational 
        posterior and the true posterior. Summing the logarithms of the variational posterior probabilities from all
          layers allows the network to capture the overall uncertainty in the model's parameters. 
        This is an essential aspect of training Bayesian Neural Networks using variational inference.
        """

        return x, log_prior, log_variational_posterior

    def predict_probabilities(self, x: torch.Tensor, num_mc_samples: int = 100) -> torch.Tensor:
        """
        Predict class probabilities for the given features by sampling from this BNN.

        :param x: Features to predict on, float tensor of shape (batch_size, in_features)
        :param num_mc_samples: Number of MC samples to take for prediction
        :return: Predicted class probabilities, float tensor of shape (batch_size, 10)
            such that the last dimension sums up to 1 for each row
        """
        probability_samples = torch.stack([F.softmax(self.forward(x)[0], dim=1) for _ in range(num_mc_samples)], dim=0)
        estimated_probability = torch.mean(probability_samples, dim=0)

        assert estimated_probability.shape == (x.shape[0], 10)
        assert torch.allclose(torch.sum(estimated_probability, dim=1), torch.tensor(1.0))
        return estimated_probability

In [ ]:
class Model(object):
    """
    BNN using Bayes by backprop
    """

    def __init__(self):
        # Hyperparameters and general parameters
        self.num_epochs = 20  # number of training epochs
        self.batch_size = 128  # training batch size
        learning_rate = 1e-3  # training learning rates
        hidden_layers = (400, 400)  # for each entry, creates a hidden layer with the corresponding number of units
        self.print_interval = 100  # number of batches until updated metrics are displayed during training
        self.n_mcs = 1 # Number of monte-carlo samples

        # BayesNet
        print('Using a BayesNet model')
        self.network = BayesNet(in_features=28 * 28, hidden_features=hidden_layers, out_features=10)

        # Optimizer for training
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=learning_rate)